In [ ]:

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

In [ ]:
dataset = pd.read_csv('Load Profile & DSM.csv')

In [ ]:
dataset

In [ ]:
# Initialize an empty list to store extracted values
load = []
scheduled = []
ODUD = []
# Iterate over every third column (starting from the second column)
for col_idx1 in range(1, dataset.shape[1], 3):
    # Extract values from the current column, starting from the fourth row
    column_values1 = dataset.iloc[2:, col_idx1].values.tolist()
    # Append the extracted values to the list
    load.extend(column_values1)

load = np.array(load)

print("Shape of load:", load.shape)
print(load)

for col_idx2 in range(2, dataset.shape[1], 3):
    column_values2 = dataset.iloc[2:, col_idx2].values.tolist()
    scheduled.extend(column_values2)
scheduled = np.array(scheduled)

print("Shape of scheduled:",scheduled.shape)
print(scheduled)

for col_idx3 in range(3, dataset.shape[1], 3):
    column_values3 = dataset.iloc[2:, col_idx3].values.tolist()
    ODUD.extend(column_values3)
ODUD = np.array(ODUD)

print("Shape of ODUD:",ODUD.shape)
print(ODUD)

In [ ]:
load

In [ ]:
data_numeric = pd.to_numeric(load, errors='coerce')
load=np.array(load, dtype=float)
data_numeric = pd.to_numeric(scheduled, errors='coerce')
scheduled=np.array(scheduled, dtype=float)
data_numeric = pd.to_numeric(ODUD, errors='coerce')
ODUD=np.array(ODUD, dtype=float)
# Check the data type after conversion
print(load.dtype)
print(scheduled)
load

In [ ]:
load[0:96].max()

In [ ]:

df = pd.DataFrame({'Load': load, 'Scheduled': scheduled, 'OD/UD': ODUD})
df.shape

In [ ]:
missing_values = df.isnull().sum()
print("Missing Values:")
print(missing_values)
#print(len(missing_values))

In [ ]:
import pandas as pd

# Your original DataFrame (df) with Load, Scheduled, and OD/UD columns
# Assuming df is already defined

# Define the start and end dates
start_date = '2022-04-01 00:00:00'
end_date = '2023-03-31 00:00:00'

# Create a DatetimeIndex with 15-minute frequency
date_range = pd.date_range(start=start_date, end=end_date, freq='15T')

# Create a DataFrame with the dates
df_dates = pd.DataFrame({'timestamp': date_range})

# Change the format of the timestamp to day-month-2023
df_dates['timestamp'] = df_dates['timestamp'].dt.strftime('%d-%m-%Y %H:%M:%S')

# Insert the timestamp column at the beginning of the DataFrame
df.insert(0, 'timestamp', df_dates['timestamp'])

# Display the updated DataFrame
print(df)


In [ ]:
df.to_csv('data_extraction.csv', index=False)

In [ ]:
# from google.colab import files

# output_csv_path = 'arhf.csv'

# files.download(output_csv_path)


In [ ]:

df['timestamp'] = pd.to_datetime(df['timestamp'], format='%d-%m-%Y %H:%M:%S')

weekday_dummies = pd.get_dummies(df['timestamp'].dt.dayofweek, prefix='weekday')

weekday_dummies.columns = ['weekday_' + str((i + 1) % 7) for i in range(7)]

df = pd.concat([df, weekday_dummies], axis=1)
#df.drop(columns=['timestamp'], inplace=True)

print(df)


In [ ]:
df.head()

In [ ]:
df

In [ ]:

lag96 = np.zeros([len(load),])
lag96[96:len(load)]=load[0:len(load)-96]
print(lag96)
print(lag96.shape)

lag192 = np.zeros([len(load),])
lag192[96*2:len(load)]=load[0:len(load)-96*2]
print(lag192)
print(lag192.shape)

lag672 = np.zeros([len(load),])
lag672[96*7:len(load)]=load[0:len(load)-96*7]
print(lag672)
print(lag672.shape)

In [ ]:
df['Lag96'] = lag96
df['Lag192'] = lag192
df['Lag672'] = lag672

In [3]:
df

NameError: name 'df' is not defined

In [ ]:
#df.to_csv('Predictive_matrix.csv', index=False)


In [ ]:
time=df.iloc[:,0].values

In [ ]:
n = len(time) - 960

time_some = time[:n]
time_remaining = time[n:]

print("time_some shape:", time_some.shape)
print("time_remaining shape:", time_remaining.shape)


In [ ]:
X = df.iloc[:, 4:].values
y = df.iloc[:,1].values

In [ ]:
z = df.iloc[:,2].values

In [ ]:

n = len(z) - 960

z_some = z[:n]
z_remaining = z[n:]

print("z_some shape:", z_some.shape)
print("z_remaining shape:", z_remaining.shape)


In [ ]:
z_some

In [ ]:
# Get the names of columns for X
X_columns = df.columns[4:]

# Get the name of the column for y
y_column = df.columns[1]

print("Columns for X:", X_columns)
print("Column for y:", y_column)


In [ ]:
X

Splitting the dataset into the Training set and Test set

In [ ]:
training_set = df.iloc[:, 1].values
# training_set=training_set.reshape(-1,1)
# training_set.shape
# Feature Scaling
# from sklearn.preprocessing import MinMaxScaler
# sc = MinMaxScaler(feature_range = (0, 1))
# training_set_scaled = sc.fit_transform(training_set)
training_set=training_set.reshape(-1,1)
# Creating a data structure with 60 timesteps and 1 output
X_train = []
y_train = []
for i in range(7, 31776):
    X_train.append(training_set[i-7:i, 0])
    y_train.append(training_set[i, 0])
X_train
X_train, y_train = np.array(X_train), np.array(y_train)
X_train.shape
y_train.shape
# Reshaping
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [ ]:
y_train.dtype

In [ ]:
X_train.shape
# y_train.shape

In [ ]:

m = len(time) - 960

load_train = load[:m]
load_test = load[m:]

print("Load_train shape:",load_train.shape)
print("Load_test shape:", load_test.shape)




load_test = load_test.reshape(-1,1)

X_test = []
y_test = []
for i in range(7, 960):
    X_test.append(load_test[i-7:i, 0])
    y_test.append(load_test[i, 0])
X_test, y_test = np.array(X_test), np.array(y_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
X_test.shape
y_test.shape


In [ ]:
import tensorflow as tf
import tensorflow_probability as tfp
import keras
from tensorflow_probability import distributions as tfd
from tensorflow.keras.layers import Input, Dense, Activation, Concatenate
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
keras.backend.clear_session()

import matplotlib.gridspec as grid_spec

In [ ]:
class MDN(tf.keras.Model):

    def __init__(self, neurons=100, components = 2, timesteps=7, features=1):
        super(MDN, self).__init__(name="MDN")
        self.neurons = neurons
        self.components = components

        self.h1 = LSTM(neurons, activation='relu', input_shape=(timesteps, features))
        self.h2 = Dense(neurons, activation="relu", name="h2")

        self.alphas = Dense(components, activation="softmax", name="alphas")
        self.mus = Dense(components, name="mus")
        self.sigmas = Dense(components, activation="nnelu", name="sigmas")
        self.pvec = Concatenate(name="pvec")

    def call(self, inputs):
        x = self.h1(inputs)
        x = self.h2(x)

        alpha_v = self.alphas(x)
        mu_v = self.mus(x)
        sigma_v = self.sigmas(x)

        return self.pvec([alpha_v, mu_v, sigma_v])

In [ ]:
def nnelu(input):
    """ Computes the Non-Negative Exponential Linear Unit
    """
    return tf.add(tf.constant(1, dtype=tf.float32), tf.nn.elu(input))


tf.keras.utils.get_custom_objects().update({'nnelu': Activation(nnelu)})

In [ ]:
def slice_parameter_vectors(parameter_vector):
    """ Returns an unpacked list of parameter vectors.
    """
    return [parameter_vector[:,i*components:(i+1)*components] for i in range(no_parameters)]

In [ ]:
def gnll_loss(y, parameter_vector):
    """ Computes the mean negative log-likelihood loss of y given the mixture parameters.
    """
    alpha, mu, sigma = slice_parameter_vectors(parameter_vector) # Unpack parameter vectors

    gm = tfd.MixtureSameFamily(
        mixture_distribution=tfd.Categorical(probs=alpha),
        components_distribution=tfd.Normal(
            loc=mu,
            scale=sigma))

    log_likelihood = gm.log_prob(tf.transpose(y)) # Evaluate log-probability of y

    return -tf.reduce_mean(log_likelihood, axis=-1)

In [ ]:
print(y_test.shape)
y_train.shape

In [ ]:
print("X_train shape:",X_train.shape)
print("X_test shape:", X_test.shape)

In [ ]:
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

In [ ]:
no_parameters = 3

components = 2
neurons = 200

opt = tf.optimizers.Adam(1e-3)

In [ ]:
mdn = MDN(neurons=neurons, components=components, timesteps=7, features=1)
mdn.compile(loss=gnll_loss, optimizer=opt)

In [ ]:
X_train.shape

In [ ]:
%%time
#mdn.fit(x=x_train, y=y_train, epochs=1000, validation_data=(x_val, y_val), callbacks=[tensorboard], batch_size=128, verbose=0)
mdn.fit(x=X_train, y= y_train, epochs=2, validation_data=(X_test, y_test), batch_size=32, verbose=1)

In [ ]:
y_pred = mdn.predict(X_test, verbose=0)
alpha_pred, mu_pred, sigma_pred = slice_parameter_vectors(y_pred)

In [ ]:
print(alpha_pred)
print(mu_pred)
print(sigma_pred)

In [ ]:
print(alpha_pred.shape)
print(mu_pred.shape)
print(sigma_pred.shape)

In [ ]:
y_pred.shape

In [ ]:
mu_pred

In [ ]:
y_pred[0][1]+y_pred[0][0]

In [ ]:
y_pred

In [ ]:
history = mdn.fit(X_train, y_train, epochs=2, validation_data=(X_test, y_test), batch_size=32, verbose=1)
nll = history.history['loss'][-1]  # Access last epoch's NLL

In [ ]:
nll

In [ ]:

def evaluate_and_visualize(mdn, X_test, y_test, num_samples=1000):
    # Make predictions
    y_pred = mdn.predict(X_test)
    alpha_pred, mu_pred, sigma_pred = slice_parameter_vectors(y_pred)

    # Sample and plot distributions for a few examples
    for i in range(3):  # Plot for 3 test samples
        # Sample from predicted mixture
        samples = tf.random.normal(shape=(num_samples, n_components), mean=mu_pred[i], stddev=sigma_pred[i])

        # Plot samples and predicted distributions
        plt.hist(samples.numpy().flatten(), bins=50, density=True, label="Sampled Points")
        x = np.linspace(mu_pred[i] - 3 * sigma_pred[i], mu_pred[i] + 3 * sigma_pred[i], 100)  # Adjust range for visualization

        # Assuming each component is Gaussian, plot individual Gaussian distributions
        for k in range(n_components):
            component_pdf = tfd.Normal(loc=mu_pred[i][k], scale=sigma_pred[i][k]).prob(x)
            plt.plot(x, component_pdf, label=f"Component {k+1}")

        plt.title(f"Predicted Mixture Distribution (Sample {i+1})")
        plt.xlabel("Value")
        plt.ylabel("Density")
        plt.legend()
        plt.show()

evaluate_and_visualize(mdn, X_test, y_test)

In [ ]:
def plot_prediction_intervals_mdn(mu_pred, sigma_pred, ts_Y_train, ts_Y_test):
  """
  This function plots the predictions from an MDN model, including:
  - Training data (blue circles)
  - Testing data (gray circles)
  - Point forecasts (one for each component, black and orange lines)
  - Confidence intervals (one for each component, shaded areas)

  Args:
      mu_pred: NumPy array of predicted means (num_samples, n_components, forecast_h).
      sigma_pred: NumPy array of predicted standard deviations (num_samples, n_components, forecast_h).
      ts_Y_train: Time series training data.
      ts_Y_test: Time series testing data.
  """

  forecast_h = len(mu_pred[:, 0])
  train_window = len(ts_Y_train)
  x_length = forecast_h + train_window
  aa = [x for x in range(x_length)]

  plt.figure(figsize=(20, 8))

  # Plot training and testing data
  plt.plot(aa[:train_window], ts_Y_train, marker='o', label="Train", color='blue')
  plt.plot(aa[train_window:], ts_Y_test, marker='o', label="Test", alpha=0.3, color='gray')

  # Plot point forecasts (one for each component)
  plt.plot(aa[train_window:], mu_pred[:, 0].reshape(-1), label="Forecast (Component 1)", color='black', linewidth=2)
  plt.plot(aa[train_window:], mu_pred[:, 1].reshape(-1), label="Forecast (Component 2)", color='orange', linewidth=2)

  # Plot confidence intervals (one for each component)
  mean_1, std_1 = mu_pred[:, 0].reshape(-1), np.sqrt(sigma_pred[:, 0])**2
  plt.errorbar(aa[train_window:], mean_1, yerr=std_1, color='red', fmt='o', label="Confidence Interval (Component 1)")
  plt.fill_between(aa[train_window:], mean_1 - std_1, mean_1 + std_1, color='b', alpha=0.2)

  mean_2, std_2 = mu_pred[:, 1].reshape(-1), np.sqrt(sigma_pred[:, 1])**2
  plt.errorbar(aa[train_window:], mean_2, yerr=std_2, color='red', fmt='o', label="Confidence Interval (Component 2)")
  plt.fill_between(aa[train_window:], mean_2 - std_2, mean_2 + std_2, color='orange', alpha=0.2)

  # Adjust x-axis limits (optional)
  # plt.xlim(0, x_length)  # Example: Set limits to cover entire data and forecast

  # Customize plot further (e.g., titles, labels, legend)
  plt.title('MDN Model Predictions')
  plt.xlabel('Time Step')
  plt.ylabel('Value')
  plt.legend()
  plt.tight_layout()
  plt.show()

In [ ]:
plot_prediction_intervals_mdn(mu_pred, sigma_pred, y_train, y_test)

In [ ]:
mu_pred.shape

In [ ]:
 aa.dtype

In [ ]:

  forecast_h = len(mu_pred[:, 0])
  train_window = len(y_train)
  x_length = forecast_h + train_window
  aa = [x for x in range(x_length)]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_prediction_intervals_mdn(mu_pred, sigma_pred, ts_Y_train, ts_Y_test):
  """
  This function plots the predictions from an MDN model, including:
  - Training data
  - Testing data
  - Point forecasts (one for each component)
  - Confidence intervals (one for each component)

  Args:
      mu_pred: NumPy array of predicted means (num_samples, n_components, forecast_h).
      sigma_pred: NumPy array of predicted standard deviations (num_samples, n_components, forecast_h).
      ts_Y_train: Time series training data.
      ts_Y_test: Time series testing data.
  """

  forecast_h = len(mu_pred[:, 0])
  train_window = len(ts_Y_train)
  x_length = forecast_h + train_window
  aa = [x for x in range(x_length)]

  # plt.figure(figsize=(12, 8))

  # Plot training and testing data
  # plt.plot(aa[:train_window], ts_Y_train, marker='o', label="Train", color='blue')
  plt.plot(aa[train_window:], ts_Y_test,  label="Test", alpha=0.8, color='black')

  # Plot point forecasts (one for each component)
  #plt.plot(aa[train_window:], mu_pred[:, 0].reshape(-1), label="Forecast (Component 1)", color='yellow')
  plt.plot(aa[train_window:], mu_pred[:, 1].reshape(-1), label="Forecast (Component 2)", color='orange')

  # Plot confidence intervals (one for each component)
  mean_1, std_1 = mu_pred[:, 0].reshape(-1), np.sqrt(sigma_pred[:, 0])**2
  #plt.errorbar(aa[train_window:], mean_1, yerr=std_1, color='yellow',label="Confidence Interval (Component 1)")
  #plt.fill_between(aa[train_window:], mean_1 - std_1, mean_1 + std_1, color='b', alpha=0.8)

  # mean_2, std_2 = mu_pred[:, 1].reshape(-1), np.sqrt(sigma_pred[:, 1])**2
  # plt.errorbar(aa[train_window:], mean_2, yerr=std_2, color='red', fmt='o', label="Confidence Interval (Component 2)")
  # plt.fill_between(aa[train_window:], mean_2 - std_2, mean_2 + std_2, color='orange', alpha=0.2)

  # Adjust x-axis limits (optional)
  # plt.xlim(0, x_length)  # Example: Set limits to cover entire data and forecast

  # Customize plot further (e.g., titles, labels, legend)
  plt.title('MDN Model Predictions')
  plt.xlabel('Time Step')
  plt.ylabel('Value')
  plt.legend()
  plt.tight_layout()
  plt.show()


plot_prediction_intervals_mdn(mu_pred, sigma_pred, y_train, y_test)
plt.plot(y_test)


In [ ]:
first_mean_array = mu_pred[:, 1]  # Assuming mu_pred has dimensions (num_samples, n_components, forecast_h)


In [ ]:
first_mean_array.shape

In [ ]:

plt.plot(first_mean_array)

In [ ]:
y_test.shape
mu_pred[:][0]

In [ ]:

def calculate_mape(y_true, y_pred):

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    ape = np.abs((y_true - y_pred) / y_true)

    ape[np.isnan(ape)] = 0

    mape = np.mean(ape) * 100

    return mape

mape = calculate_mape(y_test, first_mean_array)
print("MAPE:", mape)


In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y_test, first_mean_array)
r2 = r2_score(y_test, first_mean_array)

print("Mean Squared Error:", mse)
print("R-squared:", r2)

In [ ]:
    def plot_prediction_intervals_mdn(mu_pred, sigma_pred, ts_Y_train, ts_Y_test):
    forecast_h = len(mu_pred[:,0])
    train_window = len(ts_Y_train)
    x_length = forecast_h+train_window
    aa=[x for x in range(x_length)]
    plt.figure(figsize=(20,8))
    plt.plot(aa[:train_window], ts_Y_train, marker='.', label="train")
    plt.plot(aa[train_window:], ts_Y_test, marker='.', label="test",  alpha=0.3)
    plt.plot(aa[train_window:], mu_pred[:,0].reshape(-1), label="forecast", color='black', linewidth=2)

    mean_1, std_1 = mu_pred[:,0].reshape(-1), np.sqrt(sigma_pred[:,0])**2
    plt.errorbar(aa[train_window:], mean_1, yerr=std_1 , color='red', fmt='.');
    plt.fill_between(aa[train_window:], mean_1 - std_1, mean_1 + std_1, color='b', alpha=0.2)

    mean_2, std_2 = mu_pred[:,1].reshape(-1), np.sqrt(sigma_pred[:,1])**2
    plt.plot(aa[train_window:], mean_2, label="forecast", color='black', linewidth=2)
    plt.errorbar(aa[train_window:], mean_2, yerr=std_2, color='red', fmt='.');
    plt.fill_between(aa[train_window:], mean_2 - std_2, mean_2 + std_2, color='b', alpha=0.2)

    plt.tight_layout()
    plt.subplots_adjust(left=0.07)
    plt.xlabel('time step', size=15)

In [ ]:
plt.plot(load_train)
plt.figure(figsize=(12, 8))
plt.show()

In [ ]:
plt.plot(load_test)
plt.show()

In [ ]:
plot_prediction_intervals_mdn(mu_pred, sigma_pred, y_train, y_test)
plt.show()

In [ ]:
plot_prediction_intervals_mdn(mu_pred, sigma_pred, y_train, y_test)
plt.xlim(350,480)
plt.show()

In [ ]:
def plot_forecast_dist(ax, index, col_index):
    mdn_alpha, mdn_mu, mdn_sigma = slice_parameter_vectors(mdn.predict(X_test[index].reshape(1,X_test[index].shape[0], -1)))

    gm = tfd.MixtureSameFamily(
            mixture_distribution=tfd.Categorical(probs=mdn_alpha),
            components_distribution=tfd.Normal(
                loc=mdn_mu,
                scale=mdn_sigma))
    pyx = gm.prob(x)
    la = "Date index {:3d}".format(index + len(y_train))
    ax.plot(x,pyx, alpha=1, color=sns.color_palette("tab10")[col_index], linewidth=2, label=la)

x = np.linspace(-20,40,int(1e3))

fig = plt.figure(figsize=(8,4), dpi=100)
ax = plt.gca()

dates_index = [10, 80]
icol = 0
for date_idx in dates_index:
    plot_forecast_dist(ax, date_idx, icol)
    icol+=1

ax.set_xlabel('y')
ax.set_ylabel("p(y|date)")
ax.legend(loc=0, borderaxespad=0.1, framealpha=1.0, fancybox=True, ncol=1, shadow=True, frameon=False, fontsize=8)

plt.show()

In [ ]:
# -----  quantiles to be predicted
QUANTILES = [0.1, 0.3, 0.5, 0.7, 0.9] # N quantiles

In [ ]:
def tilted_loss(q, y, f):
    #e = (y - f) sometimes the error is computed in this form. Quantiles are inverted
    e = (f - y)
    return keras.backend.mean(keras.backend.maximum(q * e, (q - 1) * e), axis=-1)

In [ ]:
def lstm_dqr_model(X, ts_Y_train, n_steps, n_features, q):
    # define model
    model = tf.keras.Sequential()
    model.add(layers.LSTM(50, activation='relu', input_shape=(timesteps, features)))
    model.add(layers.Dense(1))
    model.compile(optimizer='adagrad', loss=lambda y, f: tilted_loss(q, y, f))
    # fit model
    model.fit(X, ts_Y_train, epochs=4, verbose=2)
    return model

In [ ]:
num_layers=2
def lstm_dqr_model(X, ts_Y_train, n_steps, n_features, q):
    # define model
    model = tf.keras.Sequential()
    for i in range(num_layers):
        if i == 0:
            model.add(layers.LSTM(50,activation='relu', input_shape=(timesteps, features), return_sequences=True))
        else:
            model.add(layers.LSTM(32,activation='relu', return_sequences=True))
     # Access only the last time step output
    model.add(layers.Lambda(lambda x: x[:, -1, :]))  # Extract the last time step

    model.add(layers.Dense(1))
    model.compile(optimizer='adagrad', loss=lambda y, f: tilted_loss(q, y, f))
    # fit model
    model.fit(X, ts_Y_train, epochs=4, verbose=2)
    return model

In [ ]:
timesteps=7
features=1


In [ ]:
q_models = []

for i in range(0,len(QUANTILES)):
    q_models.append(lstm_dqr_model(X_train, y_train, timesteps, features, QUANTILES[i]))

In [ ]:
q_yhat = []

# predict each quantile of QUANTILES list
for i in range(0,len(QUANTILES)):
    q_yhat.append(q_models[i].predict(X_test, verbose=2))

In [ ]:
q_yhat[1][0]

In [ ]:
for i in range(5):
 print( q_yhat[i].shape)

In [ ]:
def plot_ts_forecast(y_true, q_yhat, quantiles=False, title="Forecast with Quantiles", zoom_range=None):
  if zoom_range is not None:  # Create fig and ax only for zoom plot
    fig, ax = plt.subplots(figsize=(12, 6))
  else:
    plt.figure(figsize=(12, 6))  # Create figure for non-zoom plot
    ax = plt.gca()  # Get current axis
  if zoom_range is not None:
    plt.xlim(zoom_range[0], zoom_range[1])
  plt.plot(y_true, label="True Values")
  if quantiles:
    # ... (assuming q_yhat is a list of predicted quantiles)
    for i, quantile in enumerate(QUANTILES):
      plt.plot(q_yhat[i].squeeze(), label=f"{quantile * 100}% Quantile")
  else:
    plt.plot(y_pred.squeeze(), label="Predicted Value")
  plt.xlabel("Time Step")
  plt.ylabel("Value")
  plt.title(title)
  plt.legend()
  plt.grid(True)

  # Interactive zooming with slider (optional)
  if zoom_range is not None:
    from matplotlib.widgets import Slider
    vmin, vmax = zoom_range
    slider_ax = fig.add_axes([0.2, 0.1, 0.65, 0.03])
    slider = Slider(ax=slider_ax, label='X-axis Zoom', valmin=vmin, valmax=vmax, valinit=vmin)

    def update(val):
      ax.set_xlim([val, val + (vmax - vmin)])
      fig.canvas.draw_idle()

    slider.on_changed(update)

  plt.show()

# Example usage with error checking and zoom range adjustment
def check_data_compatibility(y_true, q_yhat):
  if len(y_true) != len(q_yhat[0]):
    raise ValueError("Length mismatch between y_true and predicted quantiles!")
  for pred in q_yhat:
    if len(pred) != len(y_true):
      raise ValueError("Length mismatch within predicted quantiles!")


try:
  check_data_compatibility(y_test, q_yhat)

  zoom_range = (350, 480)  # Adjust zoom range as needed
  plot_ts_forecast(y_test, q_yhat, quantiles=True, title="Forecast with Quantiles", zoom_range=zoom_range)
except ValueError as e:
  print(f"Error: {e}")
  print("Please check your data and code for compatibility issues.")

In [ ]:

# Iterate through each quantile
for i, quantile in enumerate(QUANTILES):

    # Access the predicted values for the current quantile
    predicted_quantile = q_yhat[i][ start_index:end_index]

    # Create the plot with appropriate labels and title
    plt.plot(predicted_quantile, label=f"Predicted ({quantile})")
    plt.plot(y_test[start_index:end_index], color='red', label="Actual")
    plt.xlabel('Time')
    plt.ylabel('Load')
    plt.title(f'QLSTM Predictions for Load for 2 days (Quantile {quantile})')
    plt.legend()
    plt.show()  # Show each plot individually

In [ ]:
# Define line styles and colors (optional)
line_styles = ['-', '--', ':', '-.', ':']
colors = ['b', 'g', 'r', 'c', 'm']  # Adjust colors as needed

# Create the main plot
plt.figure(figsize=(12, 6))  # Adjust figure size as needed

# Plot actual values
plt.plot(y_test[start_index:end_index], color='red', label="Actual", linewidth=2)

# Plot for each quantile
for i, (quantile, line_style, color) in enumerate(zip(QUANTILES, line_styles, colors)):
    predicted_quantile = q_yhat[i] [start_index:end_index]
    plt.plot(predicted_quantile, label=f"Predicted ({quantile})", linestyle=line_style, color=color)

# Set labels and title
plt.xlabel('Time')
plt.ylabel('Load')
plt.title('QLSTM Predictions for Load for 2 days (All Quantiles)')
plt.legend()
plt.grid(True)  # Add grid for better readability (optional)
plt.show()

In [ ]:
# Function to plot time series forecast with predicted quantiles
def plot_ts_forecast(y_true, q_yhat, quantiles=False, title="Forecast with Quantiles", zoom_range=None):
  if zoom_range is not None:
    fig, ax = plt.subplots(figsize=(12, 6))  # Create figure and axis for zoom plot
  else:
    plt.figure(figsize=(12, 6))  # Create figure for non-zoom plot
    ax = plt.gca()  # Get current axis

  plt.plot(y_true, label="True Values")
  if quantiles:
    # Assuming q_yhat is a list of predicted quantiles (one for each model)
    for i, quantile in enumerate(QUANTILES):
      plt.plot(q_yhat[i].squeeze(), label=f"{quantile * 100}% Quantile")
  else:
    plt.plot(y_pred.squeeze(), label="Predicted Value")
  plt.xlabel("Time Step")
  plt.ylabel("Value")
  plt.title(title)
  plt.legend()
  plt.grid(True)
# Interactive zooming with slider (optional)
  if zoom_range is not None:
    from matplotlib.widgets import Slider

    # Ensure zoom_range reflects actual data range (adjust as needed)
    data_min = np.min(y_true)  # Assuming y_true holds your time series data
    data_max = np.max(y_true)
    zoom_range = (data_min - 5, data_max + 5)  # Allow some buffer around data range

  # Interactive zooming with slider (optional)
  if zoom_range is not None:
    from matplotlib.widgets import Slider

    # Define slider limits based on zoom_range
    vmin, vmax = zoom_range

    # Create slider axes (location and size)
    slider_ax = fig.add_axes([0.2, 0.1, 0.65, 0.03])

    # Create the slider object
    slider = Slider(ax=slider_ax, label='X-axis Zoom', valmin=vmin, valmax=vmax, valinit=vmin)

    def update(val):
      # Update x-axis limits based on slider value
      ax.set_xlim([val, val + (vmax - vmin)])
      # Update the plot
      fig.canvas.draw_idle()

    # Connect slider and update function
    slider.on_changed(update)

  plt.show()

# Example usage with error checking and zoom range adjustment
def check_data_compatibility(y_true, q_yhat):
  if len(y_true) != len(q_yhat[0]):
    raise ValueError("Length mismatch between y_true and predicted quantiles!")
  for pred in q_yhat:
    if len(pred) != len(y_true):
      raise ValueError("Length mismatch within predicted quantiles!")


try:
  check_data_compatibility(y_test, q_yhat)  # Check data compatibility before plotting

  # zoom_range = (5, 50)  # Adjust zoom range as needed
  plot_ts_forecast(y_test, q_yhat, quantiles=True, title="Forecast with Quantiles", zoom_range=zoom_range)
except ValueError as e:
  print(f"Error: {e}")
  print("Please check your data and code for compatibility issues.")

In [ ]:
import numpy as np
from tensorflow.keras import backend as K

# Define custom quantile loss function (Pinball Loss)
def pinball_loss(q, y, f):
  e = (f - y)  # Error calculation
  return K.mean(K.maximum(q * e, (q - 1) * e), axis=-1)

# ... (Rest of your code for training and prediction with QLSTM)

# Function to calculate coverage for a specific quantile
def calculate_coverage(y_true, q_yhat, quantile):
  # Assuming q_yhat is a list of predicted quantiles for each target quantile
  lower_bound = q_yhat[QUANTILES.index(quantile) - 1].squeeze()
  upper_bound = q_yhat[QUANTILES.index(quantile)].squeeze()
  in_range = np.logical_and(y_true >= lower_bound, y_true <= upper_bound)
  return np.mean(in_range) * 100  # Coverage as percentage

# Example usage after prediction
coverage_01 = calculate_coverage(y_test, q_yhat, 0.1)
coverage_05 = calculate_coverage(y_test, q_yhat, 0.5)
coverage_09 = calculate_coverage(y_test, q_yhat, 0.9)

print("Coverage (10th Quantile):", coverage_01, "%")
print("Coverage (Median Quantile):", coverage_05, "%")
print("Coverage (90th Quantile):", coverage_09, "%")


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Define custom loss function for probabilistic forecasting
def quantile_loss(quantiles):
    def loss(y_true, y_pred):
        losses = []
        for i, q in enumerate(quantiles):
            error = y_true - y_pred[:, i]  # Extract predictions for the current quantile
            loss_q = keras.backend.mean(keras.backend.maximum(q * error, (q - 1) * error))
            losses.append(loss_q)
        return keras.backend.mean(losses)
    return loss

# Define LSTM-based probabilistic forecasting model
def build_lstm_model(input_shape, num_units=32, num_layers=2, num_quantiles=3):
    model = keras.Sequential()
    for i in range(num_layers):
        if i == 0:
            model.add(layers.LSTM(num_units, input_shape=input_shape, return_sequences=True))
        else:
            model.add(layers.LSTM(num_units, return_sequences=True))
    model.add(layers.Dense(num_quantiles, activation='linear'))
    return model


In [ ]:

# Reshape input data
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))

# Define model architecture
input_shape = (X_train.shape[1], X_train.shape[2])
model = build_lstm_model(input_shape)

# Compile model
quantiles = np.array([0.1, 0.5, 0.9])  # Define quantiles for probabilistic forecasting
model.compile(optimizer='adam', loss=quantile_loss(quantiles))

# Train model
model.fit(X_train, y_train, epochs=2, batch_size=64, validation_split=0.2)

In [ ]:
from sklearn.model_selection import train_test_split

num_last_values = 960
total_samples = len(X)

test_indices = range(total_samples - num_last_values, total_samples)

# Split the data into train and test sets using the defined test indices
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=len(test_indices), random_state=0, shuffle=False)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

In [ ]:

print("y_test:", list(y_test))

In [ ]:
m = len(time) - 960

load_train = load[:m]
load_test = load[m:]

print("Load_train shape:",load_train.shape)
print("Load_test shape:", load_test.shape)


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split

In [ ]:
X_train.shape

In [ ]:
from tensorflow.keras.layers import BatchNormalization, Dropout

# Define the LSTM model with optimizations
model = Sequential()
model.add(LSTM(50, input_shape=(X_train.shape[1], 1), return_sequences=True))
model.add(Dropout(0.4))

model.add(LSTM((50), return_sequences=True))
model.add(Dropout(0.4))

model.add(LSTM((50), return_sequences=True))
model.add(Dropout(0.4))
model.add(LSTM(50))
model.add(Dropout(0.4))
model.add(Dense(1))

# Compile the model with a lower learning rate and use early stopping
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='mse', optimizer='adam', metrics=['mae'])

# # Implement early stopping
# early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# # Train the model with early stopping
# model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])
# Train the model with early stopping
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

In [ ]:
  # Assuming your model is already trained and you have test data X_test

# Make predictions on the test data
y_pred = model.predict(X_test)

# The predictions will be probabilities since you're using a sigmoid activation in the output layer.
# If you want binary predictions, you can round the probabilities to 0 or 1.
binary_predictions = (y_pred > 0.5).astype('int')

# Optionally, you can evaluate the performance of your model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


In [ ]:
y_pred.shape

In [ ]:
y_test.shape

In [ ]:
y_pred=y_pred.reshape(-1,)

In [ ]:
comparison = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
print(comparison)


In [ ]:
df1 = pd.DataFrame(y_test)
df2 = pd.DataFrame(y_pred)

result_df = pd.concat([df1, df2], axis=1)


In [ ]:

np.set_printoptions(precision=2)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

In [ ]:
from sklearn.metrics import mean_squared_error
# Plot original data and predictions
# plt.plot(load, label='Original Load')
plt.plot(y_pred, label="Predicted" )
plt.plot(y_test,color='red',label="Actual")
# plt.plot(np.arange(len(load), len(load)+n_steps), predictions, label='Predictions')
plt.xlabel('Time')
plt.ylabel('Load')
plt.title('Linear Regression Model Predictions for Load')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# Define the start and end indices for the portion of data to plot
start_index = 96*4
end_index = 96*6


plt.plot(y_pred[start_index:end_index], label="Predicted")
plt.plot(y_test[start_index:end_index], color='red', label="Actual")

plt.xlabel('Time')
plt.ylabel('Load')
plt.title('Linear Regression Model Predictions for Load for 2 days')
plt.legend()
plt.show()


In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)

In [ ]:
def mean_absolute_percentage_error(y_test, y_pred):
    return np.mean(np.abs((y_pred - y_test) / y_test)) * 100


mape = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Absolute Percentage Error (MAPE):", mape)



In [ ]:
def mean_absolute_percentage_error(y_test, y_pred):
    return np.mean(np.abs((y_pred - y_test) / y_test)) * 100


mape = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Absolute Percentage Error (MAPE):", mape)


In [ ]:
# Get the slope (coefficients) and intercept of the linear regression model
coefficients = model.coef_
intercept = model.intercept_

print("Coefficients:", coefficients)
print("Intercept:", intercept)

# Form the equation of the line
equation_parts = [f"{coefficients[i]:.2f}x{i+1}" for i in range(len(coefficients))]
equation = " + ".join(equation_parts) + f" + {intercept:.2f}"
print("Equation of the line:", equation)


In [ ]:
coefficients = model.coef_
intercept =model.intercept_

# Display coefficients
for i in range(len(coefficients)):
    print(f"Coefficient for x{i+1}: {coefficients[i]}")

print("Intercept:", intercept)

In [ ]:
from sklearn.metrics import mean_squared_error
# Plot original data and predictions
# plt.plot(load, label='Original Load')
plt.plot(z_remaining, label="Scheduled" )
plt.plot(y_test,color='red',label="Actual")
# plt.plot(np.arange(len(load), len(load)+n_steps), predictions, label='Predictions')
plt.xlabel('Time')
plt.ylabel('Load')
plt.title('Scheduled vs Load')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt


start_index = 96*4
end_index = 96*6

# Plot selected portion of original data and predictions
plt.plot(z_remaining[start_index:end_index], label="Scheduled")
plt.plot(y_test[start_index:end_index], color='red', label="Actual")

plt.xlabel('Time')
plt.ylabel('Load')
plt.title('Scheduled vs Load for 2 days')
plt.legend()
plt.show()


In [ ]:
def mean_absolute_percentage_error(y_test, z_remaining ):
    return np.mean(np.abs((z_remaining - y_test) / y_test)) * 100

# Assuming y_true and y_pred are the true and predicted values, respectively
mape = mean_absolute_percentage_error(y_test, z_remaining )
print("Mean Absolute Percentage Error (MAPE) for load vs scheduled:", mape)


In [ ]:

from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y_test, z_remaining)
r2 = r2_score(y_test,z_remaining)

print("Mean Squared Error:", mse)
print("R-squared:", r2)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=18, activation='relu'))
ann.add(tf.keras.layers.Dense(units=18, activation='relu'))
ann.add(tf.keras.layers.Dense(units=18, activation='relu'))
ann.add(tf.keras.layers.Dense(units=1))
ann.compile(optimizer = 'RMSProp', loss = 'mean_squared_error',metrics=['accuracy'])
ann.fit(X_train, y_train, batch_size = 32, epochs = 10)




In [ ]:
loss, accuracy = ann.evaluate(X, y)
print("Loss:", loss)
print("Accuracy:", accuracy)



In [ ]:
y_pred = ann.predict(X_test)
#np.set_printoptions(precision=2)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

In [ ]:

df3 = pd.DataFrame(y_pred)
df4 = pd.DataFrame(z_remaining)

result_df1 = pd.concat([result_df,df3, df4], axis=1)


In [ ]:

df5 = pd.DataFrame(time_remaining)

result_df2 = pd.concat([result_df1, df5], axis=1)


In [ ]:
new_column_names = ['Actual Load', 'Prediction_LR', 'Prediction_ANN', 'Scheduled', 'Timestamps']

result_df2.columns = new_column_names

print(result_df2)

In [ ]:
result_df2

In [ ]:
result_df2.to_csv('combined_data.csv', index=False)

In [ ]:
from sklearn.metrics import mean_squared_error
# Plot original data and predictions
# plt.plot(load, label='Original Load')
plt.plot(y_pred, label="Predicted" )
plt.plot(y_test,color='red',label="Actual")
# plt.plot(np.arange(len(load), len(load)+n_steps), predictions, label='Predictions')
plt.xlabel('Time')
plt.ylabel('Load')
plt.title('ANN Model Predictions for Load')
plt.legend()
plt.show()


In [ ]:
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# Define the start and end indices for the portion of data to plot
start_index = 96*4
end_index = 96*6

# Plot selected portion of original data and predictions
plt.plot(y_pred[start_index:end_index], label="Predicted")
plt.plot(y_test[start_index:end_index], color='red', label="Actual")

plt.xlabel('Time')
plt.ylabel('Load')
plt.title('ANN Model Predictions for Load for 2 days')
plt.legend()
plt.show()


In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)


In [ ]:
import numpy as np

def calculate_mape(y_true, y_pred):

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    ape = np.abs((y_true - y_pred) / y_true)

    ape[np.isnan(ape)] = 0

    mape = np.mean(ape) * 100

    return mape

mape = calculate_mape(y_test, y_pred)
print("MAPE:", mape)


In [ ]:
import numpy as np

def calculate_mae(y_true, y_pred):

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    absolute_errors = np.abs(y_true - y_pred)

    mae = np.mean(absolute_errors)

    return mae

mae = calculate_mae(y_test, y_pred)
print("MAE:", mae)


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Root Mean Squared Error (RMSE)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error (RMSE):", rmse)

# Mean Absolute Percentage Error (MAPE)
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mape = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Absolute Percentage Error (MAPE):", mape)


In [ ]:

value = 13.676

indices = np.where(load == value)

print("Indices of value", value, ":", indices[0])

In [ ]:
load[-193]

In [ ]:


from statsmodels.tsa.stattools import adfuller

def check_stationarity(series):
    result = adfuller(series)

    print('ADF Statistic: %f' % result[0])
    print('p-value: %f' % result[1])
    print('Critical Values:')
    for key, value in result[4].items():
        print('\t%s: %.3f' % (key, value))

    if (result[1] <= 0.05) & (result[4]['5%'] > result[0]):
        print("\u001b[32mStationary\u001b[0m")
    else:
        print("\x1b[31mNon-stationary\x1b[0m")


In [ ]:
check_stationarity(load)

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
import matplotlib.pyplot as plt
width = 10
height = 6
lag_acf = 170
lag_pacf = 170
f, ax = plt.subplots(nrows=2, ncols=1, figsize=(width, 2*height))

# Plot ACF
plot_acf(load, lags=lag_acf, ax=ax[0])

# Plot PACF
plot_pacf(load, lags=lag_pacf, ax=ax[1], method='ols')

ax[1].annotate('Strong correlation at lag = 1', xy=(1, 0.6), xycoords='data',
               xytext=(0.17, 0.75), textcoords='axes fraction',
               arrowprops=dict(color='red', shrink=0.05, width=1))

plt.tight_layout()
plt.show()

In [ ]:
# Create subplots
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(10, 10))

# Plot PACF on the first subplot
plot_pacf(load, ax=ax1)
ax1.set_title('Partial Autocorrelation Function (PACF)')

# Plot ACF on the second subplot
plot_acf(load, ax=ax2)
ax2.set_title('Autocorrelation Function (ACF)')

plt.tight_layout()
plt.show()

In [ ]:
load

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(load)
plt.title('Load Data')
plt.xlabel('Quarterly Time')
plt.ylabel('Load in MW')
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(load[:700])  # Plotting the first 1000 data points
plt.title('Load Data (First 1000 Data Points)')
plt.xlabel('Quarterly Time')
plt.ylabel('Load in MW')
plt.grid(True)
plt.show()

In [ ]:

# Assuming 'load' is your NumPy array containing time series data
dates = pd.date_range(start='2022-01-01', periods=len(load), freq='15min')
load_df = pd.DataFrame({'Value': load}, index=dates)

# Convert index to DatetimeIndex
load_df.index = pd.to_datetime(load_df.index)
load_df

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose

# Assuming 'load' is your time series data
# Perform seasonal decomposition
decomposition = seasonal_decompose(load, model='additive', period=12)  # Adjust period as needed

# Plot the original time series and its components
plt.figure(figsize=(12, 8))

plt.subplot(411)
plt.plot(load, label='Original')
plt.legend()

plt.subplot(412)
plt.plot(decomposition.trend, label='Trend')
plt.legend()

plt.subplot(413)
plt.plot(decomposition.seasonal, label='Seasonal')
plt.legend()

plt.subplot(414)
plt.plot(decomposition.resid, label='Residual')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
# Multiplicative Decomposition
plt.figure(figsize=(12, 8))
decomp_mul = seasonal_decompose(df['Load'], model='multiplicative', extrapolate_trend='freq', period=365)
decomp_mul.plot()
plt.show()

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
# Multiplicative Decomposition
decomp_mul = seasonal_decompose(df['Load'], model='additive', extrapolate_trend='freq', period=365)
decomp_mul.plot()
plt.show()

Split dataset load into Train and Test:
Testing for last 7 days

In [ ]:
train=load[:len(load)-100]
test=load[len(load)-100:]

In [ ]:
from statsmodels.tsa.ar_model import AutoReg

# Fit AR model
model = AutoReg(train, lags=8).fit() # Example: AR model with lag order 2
print(model.summary())



In [ ]:

pred=model.predict(start=len(train),end=len(load)-1,dynamic=False)

In [ ]:
comparison = pd.DataFrame({'Actual': test, 'Predicted': pred})
print(comparison)


In [ ]:
from sklearn.metrics import mean_squared_error
# Plot original data and predictions
# plt.plot(load, label='Original Load')
plt.plot(pred)
plt.plot(test,color='red')
# plt.plot(np.arange(len(load), len(load)+n_steps), predictions, label='Predictions')
plt.xlabel('Time')
plt.ylabel('Load')
plt.title('AR Model Predictions for Load')
plt.legend()
plt.show()

# Calculate RMSE (optional)
# test_data = np.random.randn(n_steps)  # Example: test data
rmse = np.sqrt(mean_squared_error(test, pred))
print('Root Mean Squared Error (RMSE):', rmse)

In [ ]:
pred_future=model.predict(start=len(load)+1,end=len(load)+7,dynamic=False)
print(pred_future)



In [ ]:
from statsmodels.tsa.arima.model import ARIMA
# Fit a Moving Average (MA) model with order 1, discarding initial observations
model = ARIMA(train, order=(0, 0, 5)).fit()  # ARIMA(p=0, d=0, q=1) for MA(1)
print(model.summary())

In [ ]:
predMA=model.predict(start=len(train),end=len(load)-1,dynamic=False)

In [ ]:
comparison = pd.DataFrame({'Actual': test, 'Predicted': predMA})
print(comparison)
from sklearn.metrics import mean_squared_error
# Plot original data and predictions
# plt.plot(load, label='Original Load')
plt.plot(predMA)
plt.plot(test,color='red')
# plt.plot(np.arange(len(load), len(load)+n_steps), predictions, label='Predictions')
plt.xlabel('Time')
plt.ylabel('Load')
plt.title('AR Model Predictions for Load')
plt.legend()
plt.show()

# Calculate RMSE (optional)
# test_data = np.random.randn(n_steps)  # Example: test data
rmse = np.sqrt(mean_squared_error(test, predMA))
print('Root Mean Squared Error (RMSE):', rmse)

In [ ]:
print(model.summary())

In [ ]:
!pip install pmdarima

In [ ]:
from pmdarima.arima import auto_arima

In [ ]:
model_arima = auto_arima(train, seasonal=False, trace=True)

# Print the summary of the best model
print(model_arima.summary())

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
# Fit a Moving Average (MA) model with order 1, discarding initial observations
modelARIMA = ARIMA(train, order=(5, 0, 5)).fit()  # ARIMA(p=0, d=0, q=1) for MA(1)
print(modelARIMA.summary())

In [ ]:
predARIMA=modelARIMA.predict(start=len(train),end=len(load)-1,dynamic=False)

In [ ]:
comparison = pd.DataFrame({'Actual': test, 'Predicted': predARIMA})
print(comparison)
from sklearn.metrics import mean_squared_error
# Plot original data and predictions
# plt.plot(load, label='Original Load')
plt.plot(test, label='Actual Load')  # Plot the actual load
plt.plot(predARIMA, color='red', label='Predicted Load')  # Plot the predicted load
# plt.plot(np.arange(len(load), len(load)+n_steps), predictions, label='Predictions')
plt.xlabel('Time')
plt.ylabel('Load')
plt.title('AR Model Predictions for Load')
plt.legend()
plt.show()

# Calculate RMSE (optional)
# test_data = np.random.randn(n_steps)  # Example: test data
rmse = np.sqrt(mean_squared_error(test, predARIMA))
print('Root Mean Squared Error (RMSE):', rmse)

In [ ]:
model_arima = auto_arima(train, seasonal=False,start_p=1, d=0, start_q=1,  # Lower values for p and q
                         max_p=5, max_q=5,              # Upper limit for p and q
                         information_criterion='aic',   # Prioritize faster model selection
                          trace=True)

# Print the summary of the best model
print(model_arima.summary())

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
# Fit a Moving Average (MA) model with order 1, discarding initial observations
modelARIMA = ARIMA(train, order=(5, 0, 5)).fit()  # ARIMA(p=0, d=0, q=1) for MA(1)
print(modelARIMA.summary())

In [ ]:
predARIMA=modelARIMA.predict(start=len(train),end=len(load)-1,dynamic=False)
comparison = pd.DataFrame({'Actual': test, 'Predicted': predARIMA})
print(comparison)
from sklearn.metrics import mean_squared_error
plt.plot(test, label='Actual Load')  # Plot the actual load
plt.plot(predARIMA, color='red', label='Predicted Load')  # Plot the predicted load
# plt.plot(np.arange(len(load), len(load)+n_steps), predictions, label='Predictions')
plt.xlabel('Time')
plt.ylabel('Load')
plt.title('AR Model Predictions for Load')
plt.legend()
plt.show()

rmse = np.sqrt(mean_squared_error(test, predARIMA))
print('Root Mean Squared Error (RMSE):', rmse)